# 抽出型文書要約

## 手続き

1. 前準備
1. クリーニング(文区切り，絵文字/記号などの削除)
1. 文章の正規化(全角半角変換，数字の削除)
1. 文章の単語分割(形態素解析)
1. 単語の正規化(同じ意味を持つ単語の統一)
1. ストップワードの除去(名詞/形容詞/副詞/動詞のみの利用，辞書による除去)
1. 文書のベクトル表現(TF-IDF)
1. 要約モデルの適用(LexRank)
1. FlaskでAPIに

## 実装

### 1. 前準備

In [282]:
import numpy as np
import pandas as pd

import neologdn

# 文区切り
import functools
from ja_sentence_segmenter.common.pipeline import make_pipeline
from ja_sentence_segmenter.concatenate.simple_concatenator import concatenate_matching
from ja_sentence_segmenter.normalize.neologd_normalizer import normalize
from ja_sentence_segmenter.split.simple_splitter import split_newline, split_punctuation

# クリーニング
import re
from experiment.modules import data_cleaning as dc

# MeCab(形態素解析)
import MeCab
tagger = MeCab.Tagger('-r /etc/mecabrc -d /usr/lib/x86_64-linux-gnu/mecab/dic/mecab-ipadic-neologd')

# 単語の正規化
import emoji
import mojimoji

# ストップワード除去
# 名詞/形容詞/副詞/動詞のみを抽出できるライブラリのインポート
import mecabpr
mpr = mecabpr.MeCabPosRegex('-r /etc/mecabrc -d /usr/lib/x86_64-linux-gnu/mecab/dic/mecab-ipadic-neologd')

# 抽出型要約モデル(LexRank)
from sumy.parsers.plaintext import PlaintextParser
from sumy.nlp.tokenizers import Tokenizer
from sumy.summarizers.lex_rank import LexRankSummarizer

### 2. クリーニング

In [436]:
split_punc2 = functools.partial(split_punctuation, punctuations=r"。!?")
concat_tail_te = functools.partial(concatenate_matching, former_matching_rule=r"^(?P<result>.+)([\r\n]+)$", remove_former_matched=False)
segmenter = make_pipeline(normalize, split_newline, concat_tail_te, split_punc2)

text_org = """
島のアジが釣れないまま真夏のシーズンオフへ！
アジがオフでもケンサキイカシーズンはこれからです。
アジが外れ年で長らく島に行けませんでしたが、ケンサキイカはとりあえず行ってみるしかありません。
まさかのケンサキイカも外れ年で釣れないって事は無いだろうな。
勿論自然の事だから分かりませんが、誰かが確かめに行かねばなりません。
アジが良く釣れた昨年でも、盆過ぎになるとアジは３匹しか釣れませんでした。
アジは本気で狙っていなかった事もあるが、ケンサキイカも２３杯と調子が悪かった。
今年はアジがダメなのは分かっているので、ケンサキイカだけでも一昨年並みに５０杯以上釣れると良いのだが。
んんん〜〜〜楽しみ〜〜〜！
"""

In [437]:
# 『楽しみ〜〜〜』の『〜〜〜』などを削除
text = neologdn.normalize(text_org)

In [438]:
sentences = list(segmenter(text))
pd.DataFrame(sentences)

,0
0,島のアジが釣れないまま真夏のシーズンオフへ!
1,アジがオフでもケンサキイカシーズンはこれからです。
2,アジが外れ年で長らく島に行けませんでしたが、ケンサキイカはとりあえず行ってみるしかありません。
3,まさかのケンサキイカも外れ年で釣れないって事は無いだろうな。
4,勿論自然の事だから分かりませんが、誰かが確かめに行かねばなりません。
5,アジが良く釣れた昨年でも、盆過ぎになるとアジは3匹しか釣れませんでした。
6,アジは本気で狙っていなかった事もあるが、ケンサキイカも23杯と調子が悪かった。
7,今年はアジがダメなのは分かっているので、ケンサキイカだけでも一昨年並みに50杯以上釣れると良...
8,んんん楽しみ!


### 3. 文章の正規化

In [439]:
sentences_dc = dc.data_cleaning(sentences)
pd.DataFrame(sentences_dc)

,0
0,島のアジが釣れないまま真夏のシーズンオフへ
1,アジがオフでもケンサキイカシーズンはこれからです。
2,アジが外れ年で長らく島に行けませんでしたが、ケンサキイカはとりあえず行ってみるしかありません。
3,まさかのケンサキイカも外れ年で釣れないって事は無いだろうな。
4,勿論自然の事だから分かりませんが、誰かが確かめに行かねばなりません。
5,アジが良く釣れた昨年でも、盆過ぎになるとアジは0匹しか釣れませんでした。
6,アジは本気で狙っていなかった事もあるが、ケンサキイカも0杯と調子が悪かった。
7,今年はアジがダメなのは分かっているので、ケンサキイカだけでも一昨年並みに0杯以上釣れると良い...
8,んんん楽しみ


### 4.5.6. 文章の単語分割，単語の正規化，ストップワード除去

In [440]:
sentence_words = []
result = []
for i, sentence in enumerate(sentences_dc):
    # 名詞|形容詞|副詞|動詞のみの抽出
    ma = sum(mpr.findall(sentence, "(名詞|形容詞|副詞|動詞)", raw=True), [])
    # 活用形の統一(基本形へ)
    sentence_ma = []
    for word in ma:
        if word.split(',')[6] == "*":
            sentence_ma.append(word.split('\t')[0])
        else:
            sentence_ma.append(word.split(',')[6])
    result.append(sentence_ma)

In [441]:
result

[['島', 'アジ', '釣れる', 'まま', '真夏', 'シーズンオフ'],
 ['アジ', 'オフ', 'ケンサキイカ', 'シーズン', 'これから'],
 ['アジ', '外れる', '年', '長らく', '島', '行ける', 'ケンサキイカ', 'とりあえず', '行く', 'みる', 'ある'],
 ['ケンサキイカ', '外れる', '年', '釣れる', '事', '無い'],
 ['勿論', '自然', '事', '分かる', '誰か', '確かめる', '行く', 'なる'],
 ['アジ', '良い', '釣れる', '昨年', '盆', '過ぎ', 'なる', 'アジ', '0', '匹', '釣れる'],
 ['アジ', '本気', '狙う', 'いる', '事', 'ある', 'ケンサキイカ', '0', '杯', '調子', '悪い'],
 ['今年',
  'アジ',
  'ダメ',
  'の',
  '分かる',
  'いる',
  'ケンサキイカ',
  '一昨年',
  '並み',
  '0',
  '杯',
  '以上',
  '釣れる',
  '良い',
  'の'],
 ['ん', '楽しみ']]

In [442]:
corpus = [' '.join(sentence) + u'。' for sentence in result]
"".join(corpus)

'島 アジ 釣れる まま 真夏 シーズンオフ。アジ オフ ケンサキイカ シーズン これから。アジ 外れる 年 長らく 島 行ける ケンサキイカ とりあえず 行く みる ある。ケンサキイカ 外れる 年 釣れる 事 無い。勿論 自然 事 分かる 誰か 確かめる 行く なる。アジ 良い 釣れる 昨年 盆 過ぎ なる アジ 0 匹 釣れる。アジ 本気 狙う いる 事 ある ケンサキイカ 0 杯 調子 悪い。今年 アジ ダメ の 分かる いる ケンサキイカ 一昨年 並み 0 杯 以上 釣れる 良い の。ん 楽しみ。'

In [449]:
corpus

['島 アジ 釣れる まま 真夏 シーズンオフ。',
 'アジ オフ ケンサキイカ シーズン これから。',
 'アジ 外れる 年 長らく 島 行ける ケンサキイカ とりあえず 行く みる ある。',
 'ケンサキイカ 外れる 年 釣れる 事 無い。',
 '勿論 自然 事 分かる 誰か 確かめる 行く なる。',
 'アジ 良い 釣れる 昨年 盆 過ぎ なる アジ 0 匹 釣れる。',
 'アジ 本気 狙う いる 事 ある ケンサキイカ 0 杯 調子 悪い。',
 '今年 アジ ダメ の 分かる いる ケンサキイカ 一昨年 並み 0 杯 以上 釣れる 良い の。',
 'ん 楽しみ。']

### 7. 8.文書のベクトル表現，要約モデル(Lexrank)の適用

In [516]:
# 文章要約メソッド
def summy_test(sentences_org, corpus, sum_count):
    sentences = [' '.join(sentence) + u'。' for sentence in corpus]
    text_prep = "".join(sentences)
    parser = PlaintextParser.from_string(text_prep, Tokenizer('japanese'))

    summarizer = LexRankSummarizer()
    summarizer.stop_words = ['']
    # sentences_countにて、sentence（文章の数）を選びます。
    # 例えばsentences_countを３にすると文章を3文に要約してくれます。
    summary = summarizer(document=parser.document, sentences_count=sum_count)
    
    print(u'文書要約結果')    
    b = []
    for sentence in summary:
        b.append(sentences_org[sentences.index("{}".format(sentence.__str__()))])
        b.append("\n")
    return "".join(b)

In [517]:
print(summy_test(sentences, result, sum_count=3))

文書要約結果
アジがオフでもケンサキイカシーズンはこれからです。
アジが外れ年で長らく島に行けませんでしたが、ケンサキイカはとりあえず行ってみるしかありません。
今年はアジがダメなのは分かっているので、ケンサキイカだけでも一昨年並みに50杯以上釣れると良いのだが。



In [ ]:
for sentence in summary:
        print(text[corpus.index(sentence.__str__())])